In [17]:
### 问题三
## 1.常规思路

import numpy as np
from pyqubo import Array, Constraint, Placeholder
import pandas as pd
from openjij import SQASampler
import time

# 贷款利息收益率
R = 0.08

# 约束系数 M
M = 2

# matrix shape
N1 = 10
N2 = 100

# 读取附件 csv
ht = pd.read_csv('data_100.csv') 
hv = ht.values
h1 = []
t1 = []

time1 = time.time()

# 建立 t, h Array
for o in range(hv.shape[1]):
    if (o%2 == 0):
        t1.append(hv[:,o])
    else:
        h1.append(hv[:,o])

t = np.array([np.empty(N1)]*N2)
h = np.array([np.empty(N1)]*N2)

for k in range(N2):
    for l in range(N1):
        t[k][l] = t1[k][l]
        h[k][l] = h1[k][l]

print(time.time() - time1)

time2 = time.time()

# 建立 x Array
x = Array.create('x', (N2,N1), 'BINARY')

print(time.time() - time2)


time3 = time.time()
# # 定义约束条件和目标函数
# 约束条件 1：
# 在 100 张卡中 选且仅能选 3 张信用评分卡（共计出现 3 个 1）
H1 = np.sum([(np.sum([x[i][j] for i in range(N2) for j in range(N1)]) - 3)**2])

print(time.time() - time3)

time4 = time.time()
# 约束条件 2： 
# 每张卡最多选择 1 个阈值（在每一纵列中，至多出现一个 1 ，可以全为 0）
a = []
for i in range(N2):
    for n in range(N1):
        for m in range(n+1, N1):
            a.append(x[i][n]*x[i][m])

H2 = np.sum(a)
print(time.time()-time4)


time5 = time.time()
print('计算Loss:')
# 计算 H0(Loss)
b = [(1.08 * (h[i,j] + h[p,q] + h[r,s])/3 * (t[i,j] * t[p,q] * t[r,s])\
    - R * (t[i,j] * t[p,q] * t[r,s])) * x[i][j] * x[p][q] * x[r][s] \
    for i in range(N2) for j in range(N1) for p in range(N2) for q in range(N1)for r in range(N2) for s in range(N1)]
H0 = np.sum(b) 

print(time.time() - time5)

time6 = time.time()

# 目标函数
H = H0 + Placeholder('M') * (Constraint(H1, label='H1') + Constraint(H2, label='H2'))

print(time.time()-time6)

print('建立model')
time7 = time.time()
model = H.compile()

print(time.time()- time7)
feed_dict = {'M': M}

# 求出 QUBO 矩阵
qubo, offset = model.to_qubo(feed_dict=feed_dict)

# print(qubo)


# Sample(使用 SQASampler 模拟退火)
sampler = SQASampler()

# num_reads
R = 1000

sampleset = sampler.sample_qubo(qubo, num_reads=R)

# Save to ./xx/xxx_x_xxx.csv
save_path = './save_csv/ans_3_1_{}iter_1.csv'.format(R)

save = pd.DataFrame(sampleset)
save.to_csv(save_path)

0.0
0.0005207061767578125
0.0005364418029785156
0.004159688949584961
计算Loss:
2.6114296913146973
0.0
建立model
1.1222610473632812
